In [1]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

In [7]:
gdf = gpd.read_file("data/parking_violations_2015.csv")
gdf.head()

,anon_ticket_number,issue_datetime,anon_plate_id,violation_desc,fine,issuing_agency,lat,lon,geometry
0,4674379,01/01/2015 00:00,934383,BLOCKNG MASS TRANSIT,101,SEPTA,39.97578894,-75.16347051,None
1,4707189,01/01/2015 00:00,1065037,BLOCKNG MASS TRANSIT,101,SEPTA,39.97578894,-75.16347051,None
2,4584526,01/01/2015 00:01,1262953,SIDEWALK CC,76,POLICE,39.95492679,-75.14026195,None
3,4669046,01/01/2015 00:01,47082,SIDEWALK CC,76,POLICE,39.95492679,-75.14026195,None
4,4588341,01/01/2015 00:02,1509569,SIDEWALK CC,76,POLICE,39.95492679,-75.14026195,None


In [8]:
neighborhoods = gpd.read_file("data/neighbrhoods.geojson")
neighborhoods.head()

,NAME,LISTNAME,MAPNAME,Shape_Leng,Shape_Area,NUMPOINTS,geometry
0,BRIDESBURG,Bridesburg,Bridesburg,27814.546521,4.458626e+07,149.0,"MULTIPOLYGON (((2719789.837 256235.538, 271981..."
1,BUSTLETON,Bustleton,Bustleton,48868.458365,1.140504e+08,228.0,"MULTIPOLYGON (((2733378.171 289259.945, 273281..."
2,CEDARBROOK,Cedarbrook,Cedarbrook,20021.415802,2.487174e+07,129.0,"MULTIPOLYGON (((2685267.950 279747.336, 268527..."
3,CHESTNUT_HILL,Chestnut Hill,Chestnut Hill,56394.297195,7.966498e+07,7783.0,"MULTIPOLYGON (((2678490.151 284400.400, 267851..."
4,EAST_FALLS,East Falls,East Falls,27400.776417,4.057689e+07,3715.0,"MULTIPOLYGON (((2686769.727 263625.367, 268692..."


In [9]:
gdf.crs == neighborhoods.crs

False

In [10]:
neighborhoods = neighborhoods.to_crs("EPSG:4326")

In [11]:
gdf.crs == neighborhoods.crs

False

In [12]:
location_join = gpd.sjoin(gdf, neighborhoods, op="within")

<ipython-input-12-e1effc978c34>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  location_join = gpd.sjoin(gdf, neighborhoods, op="within")


ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html

In [ ]:
location_join.head()

In [ ]:
def count_numpoints(joined, polygons, merge_on):
    grouped = joined.groupby(merge_on).size()
    df = grouped.to_frame().reset_index()
    df.columns = [merge_on, "Count"]
    return polygons.merge(df, on=merge_on, how="outer")

numpoints_per_neigh = count_numpoints(location_join, neighborhoods, "NAME")
numpoints_per_neigh.head()

In [ ]:
m = KeplerGl(height=900)
m.add_data(data=numpoints_per_neigh, name='choropleth_map')
m